<a href="https://colab.research.google.com/github/destingong/AI/blob/master/AIOps_notebook_20231104.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LangChain**

In [1]:
!pip install langchain
!pip install huggingface_hub

In [5]:
import os
from google.colab import userdata
hf_token = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_token

### Model

In [3]:
from langchain.chains import LLMChain

In [6]:
from langchain import HuggingFaceHub
llm = HuggingFaceHub(repo_id = "google/flan-t5-xxl")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
llm

### Solution Architect QA Agent
- input: user prompt
- output: string of AWS architect
- context
- instruction

#### Prompt Engineering
- PromptTemplate and ChatPromptTemplate implement the Runnable interface, the basic building block of the LangChain Expression Language (LCEL). This means they support invoke, ainvoke, stream, astream, batch, abatch, astream_log calls.

- PromptTemplate accepts a dictionary (of the prompt variables) and returns a StringPromptValue. A ChatPromptTemplate accepts a dictionary and returns a ChatPromptValue.

In [ ]:
# # [template] ChatPromptTemplate - SystemMessage, HumanMessagePromptTemplate
# from langchain.prompts import ChatPromptTemplate
# from langchain.prompts import HumanMessagePromptTemplate
# from langchain.schema.messages import SystemMessage

# chat_template = ChatPromptTemplate.from_messages(
#     [
#         SystemMessage(
#             content=(
#                 "You are a helpful assistant that re-writes the user's text to "
#                 "sound more upbeat."
#             )
#         ),
#         HumanMessagePromptTemplate.from_template("{text}"),
#     ]
# )

# prompt = chat_template.invoke({'text': "i dont like eating tasty things."}).to_string()
# prompt

#### [Template 1] Simplest PromptTemplate with User Input

In [ ]:

from langchain.prompts import PromptTemplate
question = "What is a Machine Learning web app architecture in AWS?"

template = """
    Human: {question}
    You are helpful system architect that designs AWS architecture based on user's question, based on the aspects of Compute, Network, Storage.
    In details.
    Assistant:
"""

prompt = PromptTemplate(template=template, input_variables=['question'])

#### [Template 2] Few Shot Prompt Template

In [ ]:
example = [
    {"question": "Design a QnA web app architecture in AWS?",
     "answer": """
        Step 1
        Deploy this AWS Solution into your AWS account. Open the Content Designer user interface (UI) or the Amazon Lex web client, and use Amazon Cognito to authenticate.



        Step 2
        After authentication, Amazon API Gateway and Amazon Simple Storage Service (Amazon S3) deliver the contents of the Content Designer UI.



        Step 3
        Configure questions and answers in the Content Designer. The UI sends requests to Amazon API Gateway to save the questions and answers.



        Step 4
        The Content Designer AWS Lambda function saves the input in Amazon OpenSearch Service in a question's bank index. If using text embeddings, these request pass through a machine learning (ML) model, hosted on Amazon SageMaker, to generate embeddings before being saved into the question bank on OpenSearch.

        Step 5
        Chatbot users interact with Amazon Lex through the web client UI or Amazon Connect.



        Step 6
        Amazon Lex forwards requests to the Bot Fulfillment Lambda function. Chatbot users can also send requests to this Lambda function through Amazon Alexa devices.



        Step 7
        The Bot Fulfillment Lambda function takes the user's input and uses Amazon Comprehend and Amazon Translate (if necessary) to translate non-English requests to English, and then looks up the answer in OpenSearch. If using large language model (LLM) features, such as text generation and text embeddings, these requests first pass through various ML models hosted on SageMaker. SageMaker generates the search query and embeddings to compare with those saved in the question bank on OpenSearch.



        Step 8
        If an Amazon Kendra index is configured for fallback, the Bot Fulfillment Lambda function forwards the request to Amazon Kendra if no matches were returned from the OpenSearch question bank. The text generation LLM can be used to create the search query, and to synthesize a response from the excerpts of the returned document.


     """}
]

example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")
print(example_prompt.format(**example[0]))

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=example,
    example_prompt=example_prompt,
    suffix="Question: {question}",
    input_variables=["question"]
)

#print(prompt.format(input="Who was the father of Mary Ball Washington?"))

In [ ]:
%%time
chain = prompt | llm
llm_output = chain.invoke({"question": question})
llm_output

#### [Template 3] Transformation Text to JSON

In [ ]:
template = """
    Extract all input and output AWS services mentioned in each Step from the text delimitted by three backticks below.

    Format your response as a list of JSON objects where each object has the following keys: \
    "input", "output" for each Step in text.


    As an example: \
    ['{{input: Content Designer user interface (UI), output: Amazon Cognito}}', \
     '{{input: Amazon S3, output: Content Designer user interface (UI)}}', \
     ...]

    Please return unique values.\

    Text:
    '''{text}'''

"""

In [ ]:
# template = """
#     Extract all nouns and verbs mentioned in each Step from the text delimitted by three backticks below.

#     Format your response as a list of JSON objects where each object has the following keys: \
#     "noun", "verb" for each Step in text.

#     Please return unique values.

#     As an example: \

#     ['{{noun: Amazon Cognito, verb: authenticate}}', \
#      '{{noun: Content Designer user interface (UI), verb: open}}']

#     Text:
#     '''{text}'''

# """

In [ ]:
from langchain import HuggingFaceHub

repo_list = ["lmsys/fastchat-t5-3b-v1.0",  # return one row only
             "databricks/dolly-v2-7b",  # timeout
             "google/flan-t5-base",
             "gpt2"] # return '\n'

for repo_id in repo_list:
    textToJson_model = HuggingFaceHub(repo_id = repo_id)

In [ ]:
#textToJson_model = HuggingFaceHub(repo_id = 'databricks/dolly-v2-3b')
textToJson_model = HuggingFaceHub(repo_id = 'google/flan-t5-xxl')

In [ ]:
llm_output

In [ ]:
%%time
keyword_extract_prompt = PromptTemplate(template=template, input_variables=['text'])
chain = keyword_extract_prompt | textToJson_model
keyword_output = chain.invoke({"text": llm_output})
keyword_output

In [ ]:
keyword_list = keyword_output[3:-2].split("', '")
#keyword_dict = {[i.split(",")[0].replace("noun: ", "") for i in keyword_list]:[i.split(",")[1].replace("verb: ", "") for i in keyword_list] }
keyword_list

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
keyword_list = output_parser.parse(keyword_output)

### Network Graph
- https://towardsdatascience.com/customizing-networkx-graphs-f80b4e69bedf

In [ ]:
import networkx as nx
G = nx.Graph()

for i in keyword_list:
    print(i)
    input = i.split(",")[0].replace("input: ", "")
    output = i.split(",")[1].replace("output: ", "")

    G.add_node(input)
    G.add_node(output)
    G.add_edge(input, output, weight=1.7)

nx.draw(G, with_labels = True, node_size=100)

#### [Template 4] Prompt with Different Roles
- Human message, AI message, Assistant message
- MessagePlaceholder: LangChain also provides MessagesPlaceholder, which gives you full control of what messages to be rendered during formatting. This can be useful when you are uncertain of what role you should be using for your message prompt templates or when you wish to insert a list of messages during formatting.


In [ ]:
from langchain.prompts import MessagesPlaceholder
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

human_prompt = "Show me the transformed {text}"
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([MessagesPlaceholder(variable_name="text"), human_message_template])


from langchain.schema.messages import HumanMessage, AIMessage

human_message = HumanMessage(content="Show me the transformed output")

ai_message = AIMessage(content= """
    Extract all nouns and verbs mentioned in each Step from the text delimitted by three backticks below.

    Format your response as a list of JSON objects where each object has the following keys: \
    "noun", "verb" for each Step in text.

    Please return unique values.

    As an example: \

    ['{{noun: Amazon Cognito, verb: authenticate}}', \
     '{{noun: Content Designer user interface (UI), verb: open}}']



""")

chat_prompt.format_prompt(conversation=[human_message, ai_message], text = llm_output).to_messages()

chat_prompt

In [ ]:
# SYS_PROMPT = """
#     "You are a network graph maker who extracts terms and their relations from a given context. "
#     "You are provided with a context chunk {{model_output}} Your task is to extract the ontology "
#     "of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
#     "Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n"
#         "\tTerms may include object, entity, location, organization, person, \n"
#         "\tcondition, acronym, documents, service, concept, etc.\n"
#         "\tTerms should be as atomistic as possible\n\n"
#     "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
#         "\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n"
#         "\tTerms can be related to many other terms\n\n"
#     "Thought 3: Find out the relation between each such related pair of terms. \n\n"
#     "Format your output as a list of json. Each element of the list contains a pair of terms"
#     "and the relation between them, like the follwing: \n"
#     "[\n"
#     "   {\n"
#     '       "node_1": "A concept from extracted ontology",\n'
#     '       "node_2": "A related concept from extracted ontology",\n'
#     '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
#     "   }, {...}\n"
#     "]"
# """

# USER_PROMPT = "context: ```{input}``` \n\n output: "

In [ ]:
# import spacy
# nlp = spacy.load("en_core_web_sm")
# doc = nlp(output)

# token_dict = {'node':[], 'relationship':[]}

# for index, token in enumerate(doc):
#     if token.pos_ == 'PROPN' or token.pos_ == 'NOUN':
#         token_dict['node'].append(token)
#     if token.pos_ == 'VERB':
#         token_dict['relationship'] = token
# token_dict

In [ ]:
# # output parser data structure
# from langchain.output_parsers import PydanticOutputParser
# from langchain.pydantic_v1 import BaseModel, Field, validator

# class Graph(BaseModel):
#     node: str = Field(description='entity in a sentence')
#     relationship: str = Field(description='verb in a sentence')

# parser = PydanticOutputParser(pydantic_object=Graph)

### Extract Knowledge Graph from Text Agent
- input: string of AWS architects
- output: knowledge graph components node (source, destination), edge

In [ ]:
# !pip install tiktoken

In [ ]:
# #from langchain.document_loaders import WikipediaLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.text_splitter import SentenceTransformersTokenTextSplitter
# from langchain.text_splitter import CharacterTextSplitter


# # Read the wikipedia article
# raw_documents = 'how are you'
# # Define chunking strategy
# text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=3, chunk_overlap=0
# )

# # Only take the first the raw_documents
# documents = text_splitter.split_text(raw_documents)
# documents[0]

In [ ]:
## spacy
# https://spacy.io/models

### Data Visualization

In [ ]:
# pip install pyvis

In [ ]:
# from pyvis import network as net
# net = net.Network(height='400px', width='50%', notebook=True)

# net.add_node(1)
# net.add_node(2)
# net.add_edge(1, 2)

# net.save_graph("graph.html")


In [ ]:
from IPython.core.display import HTML

display(net.show("graph.html"))

## SQL Chain

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.utilities import SQLDatabase

In [ ]:
db = SQLDatabase.from_uri("sqlite:///./Chinook.db")

In [ ]:
def get_schema(_):
    return db.get_table_info()

In [ ]:
def run_query(query):
    return db.run(query)

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [ ]:
sql_response

In [ ]:
sql_response.invoke({"question": "How many employees are there?"})

In [ ]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [ ]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | llm
)

In [ ]:
full_chain.invoke({"question": "How many employees are there?"})

In [ ]:
    AIMessage(content='There are 8 employees.', additional_kwargs={}, example=False)